In [1]:
import csv
import cv2
import numpy as np

lines = []
with open("driving_log.csv") as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

images = []
measurements = []
counter = 0
for line in lines:
    sourcepath = line[0]
    filename = sourcepath.split('\\')[-1]
    currentpath = "IMG/" + filename
    image = cv2.imread(currentpath)
    images.append(image)
    measurements.append(line[3])
    counter = counter + 1
    
print(images[0].shape)
x_train = np.array(images)
y_train = np.array(measurements)

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers.convolutional import Convolution2D

model = Sequential()
model.add(Lambda(lambda x: (x/255.0), input_shape = (160, 320, 3)))
model.add(Convolution2d(6,5,5,activation="relu"))
model.add(MaxPooling2D())
model.add(Convolution2d(6,5,5,activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.fit(x_train, y_train, validation_split=0.2, shuffle = True, epochs = 10)
model.save('model.h5')

scores = model.evaluate(x_train, y_train)
print(scores)

ImportError: No module named 'cv2'